PID Controller Notebook
* Ziegler-Nichols Method

Tutor:
* OpenAI's ChatGPT

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import odeint
from scipy.signal import find_peaks

# Function to model a first-order system (e.g., a thermal system)
def system_model(y, t, u, a, b):
    dydt = (-y + b * u) / a  # First-order system dynamics
    return dydt

# PID Controller class
class PIDController:
    def __init__(self, K_p, K_i, K_d):
        self.K_p = K_p
        self.K_i = K_i
        self.K_d = K_d
        self.prev_error = 0
        self.integral = 0

    # Calculate the PID output
    def update(self, error, dt):
        self.integral += error * dt
        derivative = (error - self.prev_error) / dt
        self.prev_error = error
        return self.K_p * error + self.K_i * self.integral + self.K_d * derivative

# System simulation with PID control
def simulate_system(K_p, K_i, K_d, t, y0, a, b):
    pid = PIDController(K_p, K_i, K_d)
    u = 0  # Control signal
    y = np.zeros_like(t)  # Output array
    y[0] = y0
    set_point = 1.0  # Desired setpoint (reference value)

    for i in range(1, len(t)):
        dt = t[i] - t[i-1]
        error = set_point - y[i-1]
        u = pid.update(error, dt)  # PID control signal
        dydt = system_model(y[i-1], t[i-1], u, a, b)  # Simplified model update
        y[i] = y[i-1] + dydt * dt  # Euler integration for system state update

    return y

# Automatically find the ultimate gain and period using the closed-loop method
def find_ultimate_gain_period(t, y):
    peaks, _ = find_peaks(y, height=0.002, distance=3)  # More sensitive peak detection
    peak_times = t[peaks]
    if len(peak_times) < 2:
        return None, None
    T_u = np.mean(np.diff(peak_times))  # Average time between peaks
    K_u = 1.0  # Manually set ultimate gain, adjust as needed
    return K_u, T_u

# Define simulation parameters
t = np.linspace(0, 100, 10000)  # Further increased time array and resolution
y0 = 0.5  # Changed initial condition
a, b = 1.5, 1.0  # System parameters

# Automatically tune K_p
def auto_tune_K_p(K_p_initial, a, b, max_K_p=100.0, increment=0.05):
    K_p = K_p_initial
    while K_p <= max_K_p:
        y_initial = simulate_system(K_p, 0, 0, t, y0, a, b)
        K_u, T_u = find_ultimate_gain_period(t, y_initial)
        if K_u is not None and T_u is not None:
            return K_p, y_initial, K_u, T_u
        K_p += increment  # Smaller increments for finer control
    return None, None, None, None

# Initial K_p value (can be adjusted automatically)
K_p_initial = 1.0
K_p_initial, y_initial, K_u, T_u = auto_tune_K_p(K_p_initial, a, b)

# Exit if tuning fails
if K_p_initial is None:
    print("Failed to find an oscillatory system response. Tuning aborted.")
else:
    # Apply Ziegler-Nichols Tuning Rules for PID
    K_p_zn = 0.6 * K_u
    K_i_zn = 2 * K_p_zn / T_u
    K_d_zn = K_p_zn * T_u / 8

    # Simulate the system with the tuned PID values
    y_tuned = simulate_system(K_p_zn, K_i_zn, K_d_zn, t, y0, a, b)

    # Output the table of PID values
    pid_table = pd.DataFrame({
        'Parameter': ['K_p', 'K_i', 'K_d'],
        'Value': [K_p_zn, K_i_zn, K_d_zn]
    })
    print("Tuned PID Parameters (Ziegler-Nichols Method):")
    print(pid_table.to_string(index=False))

    # Plot the system response before and after tuning
    plt.figure(figsize=(10, 6))
    plt.plot(t, y_initial, label=f'Before Tuning (K_p = {K_p_initial:.2f}, K_i = 0, K_d = 0)', linestyle='--')
    plt.plot(t, y_tuned, label=f'After Tuning (K_p = {K_p_zn:.2f}, K_i = {K_i_zn:.2f}, K_d = {K_d_zn:.2f})', linewidth=2)
    plt.axhline(1.0, color='gray', linestyle=':', label='Set Point')
    plt.title('System Response Before and After PID Tuning (Ziegler-Nichols Method)')
    plt.xlabel('Time')
    plt.ylabel('System Output')
    plt.legend()
    plt.grid(True)
    plt.show()


Failed to find an oscillatory system response. Tuning aborted.
